In [11]:
import pygame
import random
import sqlite3

# Initialize Pygame
pygame.init()

# Set up the game window
screen_width = 800
screen_height = 600
window = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Space Shooter")

# Game colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)


# Database class
class Database:
    def __init__(self, database):
        self.connection = sqlite3.connect(database)
        self.cursor = self.connection.cursor()

    def create_table(self):
        self.cursor.execute("CREATE TABLE IF NOT EXISTS scores (player TEXT, score INTEGER)")

    def add_player_score(self, player_name):
        self.cursor.execute("INSERT INTO scores (player) VALUES (?)", (player_name,))
        self.connection.commit()

    def update_player_score(self, player_name, score):
        self.cursor.execute("UPDATE scores SET score = ? WHERE player = ?", (score, player_name))
        self.connection.commit()

    def get_top_scores(self, limit):
        self.cursor.execute("SELECT player, score FROM scores ORDER BY score DESC LIMIT ?", (limit,))
        return self.cursor.fetchall()

    def close(self):
        self.connection.close()

# Connect to the SQLite database
database = Database("game.db")
database.create_table()

# Taking input of the player's name
font = pygame.font.Font(None, 34)
window.fill(BLACK)

input_prompt = "Enter your name: "
player_name = ""
input_rendered = font.render(input_prompt + player_name, True, WHITE)
input_rect = input_rendered.get_rect(center=(screen_width // 2, screen_height // 2))

# Input loop
progress = True
while progress:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            connection.close()
            pygame.quit()
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_RETURN:
                progress = False  # Exit the game loop when Enter is pressed
            elif event.key == pygame.K_BACKSPACE:
                player_name = player_name[:-1]
            else:
                player_name += event.unicode

    # Update the rendered input text
    input_rendered = font.render(input_prompt + player_name, True, WHITE)

    # Clear the screen
    window.fill(BLACK)

    # Draw the input text
    window.blit(input_rendered, input_rect)

    # Update the display
    pygame.display.update()
    
# Add the player's name to the database
database.add_player_score(player_name)

# Player class
class Player:
    def __init__(self):
        self.name = player_name
        self.font = pygame.font.Font(None, 27)
        self.width = 64
        self.height = 64
        self.x = (screen_width - self.width) // 2
        self.y = screen_height - self.height - 10
        self.speed = 10
        self.bullets = []
        self.max_bullets = 7
        self.shoot_interval = 300  # Interval between shots in milliseconds
        self.last_shot_time = pygame.time.get_ticks()
        
    def move_left(self):
        if self.x > 0:
            self.x -= self.speed

    def move_right(self):
        if self.x < screen_width - self.width:
            self.x += self.speed

    def shoot(self):
        current_time = pygame.time.get_ticks()
        if current_time - self.last_shot_time >= self.shoot_interval and len(self.bullets) < self.max_bullets:
            bullet = Bullet(self.x + self.width // 2, self.y)
            self.bullets.append(bullet)
            self.last_shot_time = current_time

    def update_bullets(self):
        new_bullets = []
        for bullet in self.bullets:
            bullet.update()
            if bullet.y >= 0:
                new_bullets.append(bullet)
        self.bullets = new_bullets

    def draw(self):
        name_text = self.font.render(str(self.name), True, WHITE)
        name_rect = name_text.get_rect(topleft=(27, 5))
        window.blit(name_text, name_rect)
        triangle_points = [(self.x + self.width // 2, self.y),
                           (self.x, self.y + self.height),
                           (self.x + self.width, self.y + self.height)]
        pygame.draw.polygon(window, WHITE, triangle_points)

        for bullet in self.bullets:
            bullet.draw()

# Bullet class
class Bullet:
    def __init__(self, x, y):
        self.width = 8
        self.height = 16
        self.x = x - self.width // 2
        self.y = y
        self.speed = 7

    def update(self):
        self.y -= self.speed

    def draw(self):
        pygame.draw.rect(window, WHITE, (self.x, self.y, self.width, self.height))

# Enemy class
class Enemy:
    max_enemies = 20

    def __init__(self):
        self.width = 64
        self.height = 64
        self.x = random.randint(0, screen_width - self.width)
        self.y = 27
        self.speed = 3

    def update(self):
        self.y += self.speed

    def draw(self):
        pygame.draw.rect(window, WHITE, (self.x, self.y, self.width, self.height))

    @classmethod
    def create_enemy(cls):
        if len(enemies) < cls.max_enemies:
            enemies.append(cls())

# Score counter class
class ScoreCounter:
    def __init__(self):
        self.score = 0
        self.font = pygame.font.Font(None, 27)

    def increment_score(self):
        self.score += 1

    def draw(self):
        score_text = self.font.render("Score: " + str(self.score), True, WHITE)
        score_rect = score_text.get_rect(topright=(screen_width - 10, 5))
        window.blit(score_text, score_rect)
        line_y = score_rect.bottom + 1  # Adjust the vertical position of the line
        pygame.draw.line(window, WHITE, (0, line_y), (screen_width, line_y))

# Game loop
clock = pygame.time.Clock()
running = True
player = Player()
enemies = []
enemy_spawn_timer = 0
enemy_spawn_interval = 1000  # Spawn a new enemy every 1000 milliseconds
score_counter = ScoreCounter()

while running:
    clock.tick(60)
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Player movement
    keys = pygame.key.get_pressed()
    if keys[pygame.K_LEFT]:
        player.move_left()
    if keys[pygame.K_RIGHT]:
        player.move_right()
    if keys[pygame.K_SPACE]:
        player.shoot()

    # Update player bullets
    player.update_bullets()

    # Update enemy position
    for enemy in enemies:
        enemy.update()

    # Collision detection
    new_enemies = []
    for enemy in enemies:
        remove_enemy = False
        for bullet in player.bullets:
            if bullet.x < enemy.x + enemy.width and \
                    bullet.x + bullet.width > enemy.x and \
                    bullet.y < enemy.y + enemy.height and \
                    bullet.y + bullet.height > enemy.y:
                # Handle collision logic here (e.g., score increment, enemy removal, etc.)
                score_counter.increment_score()
                remove_enemy = True
                player.bullets.remove(bullet)  # Remove the bullet after collision
                break
        if not remove_enemy:
            if enemy.y < screen_height:
                new_enemies.append(enemy)
            else:
                Enemy.create_enemy()  # Reuse the enemy if it goes off the screen
    enemies = new_enemies
    
    # Collision detection 2
    for enemy in enemies:
        enemy_rect = pygame.Rect(enemy.x, enemy.y, enemy.width, enemy.height)
        triangle_points = [(player.x + player.width // 2, player.y),
                           (player.x, player.y + player.height),
                           (player.x + player.width, player.y + player.height)]
        triangle_sides = [(triangle_points[0], triangle_points[1]),
                          (triangle_points[1], triangle_points[2]),
                          (triangle_points[2], triangle_points[0])]

        for side in triangle_sides:
            if enemy_rect.clipline(*side):
                pygame.time.wait(700)
                running = False  # Stop the game if player collides with an enemy
                break
        if not running:
            break

    # Spawn new enemies
    enemy_spawn_timer += clock.get_time()
    if enemy_spawn_timer >= enemy_spawn_interval:
        Enemy.create_enemy()
        enemy_spawn_timer = 0

    # Clear the screen
    window.fill(BLACK)

    # Draw player, enemy, bullets, and score counter
    player.draw()
    for enemy in enemies:
        enemy.draw()
    score_counter.draw()

    # Update the display
    pygame.display.update()

# Game Over message
window.fill(BLACK)
font = pygame.font.Font(None, 34)
game_over_text = font.render("Game Over", True, WHITE)
game_over_rect = game_over_text.get_rect(center=(screen_width // 2, screen_height // 2))
window.blit(game_over_text, game_over_rect)
pygame.display.update()

# Delay before displaying highscores
pygame.time.wait(1500)

# Update the player's score in the database
score = score_counter.score
database.update_player_score(player_name, score)

# Retrieve the top 7 scores from the database
top_scores = database.get_top_scores(7)

window.fill(BLACK)

# Display the scores
font1 = pygame.font.Font(None, 34)
game_over_text = font1.render("**Highscores**", True, WHITE)
game_over_rect = game_over_text.get_rect(center=(screen_width // 2, 25))
window.blit(game_over_text, game_over_rect)
font2 = pygame.font.Font(None, 25)
x = 200
y = 150
increment = 30
for i, (player, score) in enumerate(top_scores):
    rank_text = font2.render(str(i + 1), True, WHITE)
    player_text = font2.render(player, True, WHITE)
    score_text = font2.render(str(score), True, WHITE)
    window.blit(rank_text, (x, y + i * increment))
    window.blit(player_text, (x + 50, y + i * increment))
    window.blit(score_text, (x + 200, y + i * increment))
pygame.display.update()

# Delay before quitting the game
pygame.time.wait(5000)

# Quit the game
database.close()
pygame.quit()